# Notebook #3: Heat exchanger design optimization with linear and hyperplane tree surrogates

This notebook is part of the Supporting Information of the paper "Hyperplane Decision Trees as Piecewise Linear Surrogate Models for Chemical Process Design", by Sunshine et al.

Notebook #2 showed details into training surrogate models for PHT tables, using mainly the [hyperplanetree](https://github.com/LLNL/systems2atoms/tree/add-hyperplanetree/systems2atoms/hyperplanetree) Python package. In the present Notebook, we focus in showing the embeding of the surrogates in the optimization problem of a heat exchanger design. 

Contact the authors of the paper if you have any questions.

## Installing the package from github

In [ ]:
!pip install git+https://github.com/LLNL/systems2atoms

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from systems2atoms.hyperplanetree import LinearTreeRegressor, HyperplaneTreeRegressor, plot_surrogate_2d
import torch 
import sklearn
import time
from tqdm.auto import tqdm
from IPython.display import display, HTML
import base64
import io
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import lineartree
from sklearn.linear_model import LinearRegression
import matplotlib.patches as mpatches
from sklearn.metrics import mean_absolute_error

from systems2atoms.hyperplanetree import HyperplaneTreeDefinition, HyperplaneTreeHybridBigMFormulation, HyperplaneTreeGDPFormulation
from omlt.linear_tree import LinearTreeGDPFormulation, LinearTreeDefinition, LinearTreeHybridBigMFormulation

plt.rcParams['figure.dpi'] = 200
torch_device = 'cpu' 

## Preparing the data for use in the regression

In [3]:
df = pd.read_csv('PHTSV_Table_HMAX_Adjusted.csv', usecols=[0, 1, 2])

df

,P(Pa),H(J/mol),T(K)
0,1.000000e+05,1275.88638,290.000000
1,3.515152e+05,1275.88638,289.942853
2,6.030303e+05,1275.88638,289.885698
3,8.545455e+05,1275.88638,289.828536
4,1.106061e+06,1275.88638,289.771366
...,...,...,...
9995,2.399394e+07,65523.00000,920.211096
9996,2.424545e+07,65523.00000,920.898379
9997,2.449697e+07,65523.00000,921.583218
9998,2.474848e+07,65523.00000,922.265623


In [4]:
# Features will be the first two columns, pressure and enthalpy data
features = torch.tensor(df[['P(Pa)', 'H(J/mol)']].values, dtype=torch.float64)

# Response will be the third and last column, of temperature
y = torch.tensor(df['T(K)'].values, dtype=torch.float64)

features[:, 0] = features[:, 0] / 1e5  # Convert pressure from Pa to bar for consistency with plot in Ammari (20223)
features[:, 1] = features[:, 1]  / 1000  # Convert enthalpy from J/mol to kJ/mol 

pressure = features[:, 0]
enthalpy = features[:, 1]
temperature = df['T(K)']

### Getting train and test splits

Here, we use the splitting feature in [scikit-learn](https://scikit-learn.org/stable/). Alternative splitting methodologies can be used as well.

In [5]:
indices = np.arange(len(features))

# 20% of the data will be used for test, and 42 is used as the seed for random state, for reproducibility
train_features, test_features, train_y, test_y, train_indices, test_indices= train_test_split(features, y, indices, test_size=0.2, random_state=42)

# Making sure we have shuffled indices and correct splits
print(f'Train indices: {train_indices} ({np.size(train_indices)} points)')
print(f'Test indices: {test_indices} ({np.size(test_indices)} points)')

Train indices: [9254 1561 1670 ... 5390  860 7270] (8000 points)
Test indices: [6252 4684 1731 ... 7853 1095 6929] (2000 points)


## Training the different surrogates

We now train three surrogates. The first (1) is a linear model decision tree. The second (2) is a hyperplane tree model with weight = 2, and the third (3) is a hyperplane tree model with weight = 2. We use these to show how much the hyperparameters influence final results especially regarding training and optimization run times. 

### LMDT as used by [Ammari et al](https://www.sciencedirect.com/science/article/pii/S009813542300217X), with the package by [Cerliani](https://github.com/cerlymarco/linear-tree).

In [6]:
import lineartree
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


model_ltr_ammari_cerliani = lineartree.LinearTreeRegressor(LinearRegression(), 
                                                           criterion="mae", 
                                                           min_samples_leaf=0.002, 
                                                           min_impurity_decrease = -np.inf, 
                                                           max_depth = 20, 
                                                           max_bins = 50)

t1 = time.time()
model_ltr_ammari_cerliani.fit(train_features, train_y)
t2 = time.time()

y_pred = model_ltr_ammari_cerliani.predict(test_features)
linear_error = mean_absolute_error(test_y, y_pred)

linear_time = t2 - t1

tree_summary = model_ltr_ammari_cerliani.summary()

linear_leaves = sum(1 for node_info in tree_summary.values() if 'children' not in node_info)


print(f'Model                  # Leaves      Error (MAE)    Training time (s)')
print(f'Hyperplane Tree:       {linear_leaves}            {linear_error:.3f}          {linear_time:.2f}')

Current number of nodes: 336 | Current number of leaves: 337
Model                  # Leaves      Error (MAE)    Training time (s)
Hyperplane Tree:       337            0.303          3.56


### Hyperplane tree, with maximum hyperplane weight = 2

In [7]:
model_ht_2 = HyperplaneTreeRegressor(
    min_samples_leaf = 0.0043,
        max_bins = 50,
        max_weight = 2,
        disable_tqdm = True,
        min_impurity_decrease = -np.inf,
)

t1 = time.time()
model_ht_2.fit(train_features, train_y)
t2 = time.time()

y_pred_ht = model_ht_2.predict(test_features.to(torch_device)).cpu()

hyperplane_error_2 = torch.mean(torch.abs(y_pred_ht - test_y))
hyperplane_leaves_2 = len(model_ht_2._leaves)
hyperplane_time_2 = t2 - t1

print(f'Model                  # Leaves      Error (MAE)    Training time (s)')
print(f'Hyperplane Tree:       {hyperplane_leaves_2}            {hyperplane_error_2:.3f}          {hyperplane_time_2:.2f}')

Model                  # Leaves      Error (MAE)    Training time (s)
Hyperplane Tree:       174            0.277          2.90


### Hyperplane tree, with maximum hyperplane weight = 3

In [8]:
model_ht_3 = HyperplaneTreeRegressor(
    min_samples_leaf = 0.0043,
        max_bins = 50,
        max_weight = 3,
        disable_tqdm = True,
        min_impurity_decrease = -np.inf,
)

t1 = time.time()
model_ht_3.fit(train_features, train_y)
t2 = time.time()

y_pred_ht = model_ht_3.predict(test_features.to(torch_device)).cpu()

hyperplane_error_3 = torch.mean(torch.abs(y_pred_ht - test_y))
hyperplane_leaves_3 = len(model_ht_3._leaves)
hyperplane_time_3 = t2 - t1

print(f'Model                  # Leaves      Error (MAE)    Training time (s)')
print(f'Hyperplane Tree:       {hyperplane_leaves_3}            {hyperplane_error_3:.3f}          {hyperplane_time_3:.2f}')

Model                  # Leaves      Error (MAE)    Training time (s)
Hyperplane Tree:       168            0.302          7.14


## Optimization framework using the surrogates

In [9]:
import pyomo.environ as pyo
from omlt import OmltBlock

def create_model(surrogate_def, surrogate_formulation, model, **surrogate_params):
    'This function generalizes the creation of a Pyomo model that admits a surrogate model block'

    m = pyo.ConcreteModel()

    # Define parameters
    U = 6341.4
    Cp = 1.507
    Ks = 20
    Ka = 150

    # Define variables
    m.TsIn = pyo.Var()
    m.TsOut = pyo.Var()
    m.TpIn = pyo.Var()
    m.TpOut = pyo.Var()
    m.Fp = pyo.Var(within=pyo.NonNegativeReals)
    m.Fs = pyo.Var(within=pyo.NonNegativeReals)
    m.A = pyo.Var(within=pyo.NonNegativeReals)
    m.PsIn = pyo.Var(within=pyo.NonNegativeReals, bounds=(1, 250))
    m.PsOut = pyo.Var(within=pyo.NonNegativeReals, bounds=(1, 250))
    m.Q = pyo.Var(within=pyo.NonNegativeReals)
    m.HsIn = pyo.Var(within=pyo.NonNegativeReals, bounds=(1.275886, 65.523))
    m.HsOut = pyo.Var(within=pyo.NonNegativeReals, bounds=(1.275886, 65.523))
    m.LMTD = pyo.Var()
    m.dT1 = pyo.Var(bounds=(0.01, None))
    m.dT2 = pyo.Var(bounds=(0.01, None))

    # Constraints for the model - physical laws and design equations for the HEX
    m.heat_transfer = pyo.Constraint(expr=m.Q == U * m.A * m.LMTD)
    m.LMTD_chenApp = pyo.Constraint(expr=m.LMTD == (m.dT1 * m.dT2 * (m.dT1 + m.dT2) / 2) ** (1 / 3))
    m.dT1_eq = pyo.Constraint(expr=m.dT1 == m.TsOut - m.TpIn)
    m.dT2_eq = pyo.Constraint(expr=m.dT2 == m.TsIn - m.TpOut)
    m.proc_fluid_thermo = pyo.Constraint(expr=m.Q == m.Fp * Cp * (m.TpOut - m.TpIn))
    m.steam_thermo = pyo.Constraint(expr=m.Q == -m.Fs * (m.HsOut - m.HsIn) * 1000 / 18)

    # Surrogate block - this is where we embed the surrogate
    m.tree_outlet = OmltBlock()
    surrogate_instance = surrogate_def(model, **surrogate_params)
    formulation_instance = surrogate_formulation(surrogate_instance, 'hull')
    m.tree_outlet.build_formulation(formulation_instance)

    # Connect surrogate inputs and outputs
    m.connect_input_outlet_P = pyo.Constraint(expr=m.PsOut == m.tree_outlet.inputs[0])
    m.connect_input_outlet_H = pyo.Constraint(expr=m.HsOut == m.tree_outlet.inputs[1])
    m.connect_output_outlet = pyo.Constraint(expr=m.TsOut == m.tree_outlet.outputs[0])

    # Objective function - related to minimizing cost od HEX design
    m.obj = pyo.Objective(expr=Ks * m.Fs + Ka * m.A)

    # Fix variables
    m.PsIn.fix(86.047)
    m.PsOut.fix(86.047)
    m.TsIn.fix(866)
    m.TpIn.fix(513.15)
    m.TpOut.fix(831)
    m.Fp.fix(310 * 3600)
    m.HsIn.fix(65.233)

    return m

In [10]:
# Define the models
m1 = create_model(
    surrogate_def=LinearTreeDefinition,
    surrogate_formulation=LinearTreeGDPFormulation,
    model=model_ltr_ammari_cerliani,
    unscaled_input_bounds={0: (1, 250), 1: (1.275886, 65.523)}
)

m2 = create_model(
    surrogate_def=HyperplaneTreeDefinition,
    surrogate_formulation=HyperplaneTreeGDPFormulation,
    model=model_ht_2,
    input_bounds_matrix=torch.tensor([[1, 250], [1.275886, 65.523]], dtype=torch.float64)
)

m3 = create_model(
    surrogate_def=HyperplaneTreeDefinition,
    surrogate_formulation=HyperplaneTreeGDPFormulation,
    model=model_ht_3,
    input_bounds_matrix=torch.tensor([[1, 250], [1.275886, 65.523]], dtype=torch.float64)
)

# Use the global solver BARON - here we used the GAMS interface 
solver = pyo.SolverFactory("gams:baron")

# Solve m1 and measure time
start_time_m1 = time.time()
results1 = solver.solve(m1, tee=True)
end_time_m1 = time.time()
time_m1 = end_time_m1 - start_time_m1

# Solve m2 and measure time
start_time_m2 = time.time()
results2 = solver.solve(m2, tee=True)
end_time_m2 = time.time()
time_m2 = end_time_m2 - start_time_m2


# Solve m3 and measure time
start_time_m3 = time.time()
results3 = solver.solve(m3, tee=True)
end_time_m3 = time.time()
time_m3 = end_time_m3 - start_time_m3

WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[1]' to a numeric value
`0` outside the bounds (1.275886, 65.523).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[1]' to a

### Results comparison

In [11]:
# Original formulation results as mentioned in Ammari (2023)
original_obj = 4.78e6  #Value for the objective
original_Fs = 60.31   # Value for steam flow (kg/s)
original_A = 2924     # Value for heat exchanger area (m2)

m1_obj = m1.obj.expr()
m1_Fs = m1.Fs.value / 3600  # Convert to kg/s
m1_A = m1.A.value

m2_obj = m2.obj.expr()
m2_Fs = m2.Fs.value / 3600  # Convert to kg/s
m2_A = m2.A.value

m3_obj = m3.obj.expr()
m3_Fs = m3.Fs.value / 3600  # Convert to kg/s
m3_A = m3.A.value

In [12]:
comparison_df = pd.DataFrame({
    "Metric": [
        "Objective Value ($)", 
        "Steam Flow (kg/s)", 
        "Heat Exchanger Area (m2)", 
        "Run time (s)", 
        "# of leaves", 
        "MAE (K)",
        "Training time (s)",
        "BARON Memory usage (MB)"
    ],
    "Original formulation": [
        f"{original_obj:.2e}", 
        f"{original_Fs:.2f}", 
        f"{original_A:.0f}", 
        "-", 
        "-", 
        "-", 
        "-",
        "-",
    ],
    "For linear tree surrogate": [
        f"{m1_obj:.2e}", 
        f"{m1_Fs:.2f}", 
        f"{m1_A:.0f}", 
        f"{time_m1:.2f}", 
        f"{linear_leaves}", 
        f"{linear_error:.2f}",
        f"{linear_time:.2f}",
        6 
    ],
    "For hyperplane tree surrogate (W = 3)": [
        f"{m3_obj:.2e}", 
        f"{m3_Fs:.2f}", 
        f"{m3_A:.0f}", 
        f"{time_m3:.2f}", 
        f"{hyperplane_leaves_3}", 
        f"{hyperplane_error_3:.2f}",
        f"{hyperplane_time_3:.2f}",
        8 
    ],
        "For hyperplane tree surrogate (W = 2)": [
        f"{m2_obj:.2e}", 
        f"{m2_Fs:.2f}", 
        f"{m2_A:.0f}", 
        f"{time_m2:.2f}", 
        f"{hyperplane_leaves_2}", 
        f"{hyperplane_error_2:.2f}",
        f"{hyperplane_time_2:.2f}",
        11 
    ]
})

comparison_df


,Metric,Original formulation,For linear tree surrogate,For hyperplane tree surrogate (W = 3),For hyperplane tree surrogate (W = 2)
0,Objective Value ($),4.78e+06,4.76e+06,4.76e+06,4.76e+06
1,Steam Flow (kg/s),60.31,60.08,60.09,60.09
2,Heat Exchanger Area (m2),2924,2915,2919,2921
3,Run time (s),-,1.25,6.39,1.36
4,# of leaves,-,337,168,174
5,MAE (K),-,0.30,0.30,0.28
6,Training time (s),-,3.56,7.14,2.90
7,BARON Memory usage (MB),-,6,8,11


Further discussion about the results in this table can be seen in the paper.